In [1]:
!pip install torch torchvision facenet-pytorch
!pip install -U transformers


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/1.9 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 6.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.8 MB 3.3 MB/s eta 0:00:04
   ---------- ----------------------------- 2.9/10.8 MB 8.8 MB/s eta 0:00:01
   ---------- ----------------------------- 2.9/10.8 MB 8.8 MB/s eta 0:00:01
   ------------- -------------------------- 3.7/10.8 MB 5.6 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/10.8 MB 4.6 MB/s eta 0:00:02
   ---------------- ------

In [7]:
# Install dependencies
!pip install torch torchvision facenet-pytorch pandas

import torch, os, numpy as np, pandas as pd
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=160, margin=14, keep_all=False, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)


C:\Users\COMPUMARTS\anaconda3\envs\myenv\lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_di

In [15]:
CSV_PATH = r"C:\Users\COMPUMARTS\Downloads\v3\employees.csv"

def extract_embedding(img_path):
    img = Image.open(img_path).convert("RGB")  # Force RGB, drops alpha
    face = mtcnn(img)
    if face is None:
        raise ValueError("No face detected.")
    with torch.no_grad():
        embedding = resnet(face.unsqueeze(0).to(device))
    return embedding.squeeze(0).cpu().numpy()


def save_to_csv(name, embedding, csv=CSV_PATH):
    df_new = pd.DataFrame([{
      'name': name,
      'face_encoding': embedding.tolist()
    }])
    if os.path.exists(csv):
        df = pd.read_csv(csv, converters={'face_encoding': eval})
        df = pd.concat([df, df_new], ignore_index=True)
    else:
        df = df_new
    df.to_csv(csv, index=False)


In [19]:
e = extract_embedding(r"C:\Users\COMPUMARTS\Downloads\Screenshot 2025-03-23 034513.png")
save_to_csv("Alice", e)


In [21]:
def identify_person(embedding, csv=CSV_PATH, threshold=0.8):
    df = pd.read_csv(csv, converters={'face_encoding': eval})
    best_name, best_dist = "Unknown", float('inf')
    for _, row in df.iterrows():
        stored = np.array(row['face_encoding'])
        dist = np.linalg.norm(embedding - stored)
        if dist < best_dist:
            best_dist, best_name = dist, row['name']
    return best_name if best_dist < threshold else "Unknown"


In [27]:
img_path = r"C:\Users\COMPUMARTS\Downloads\WhatsApp Image 2023-08-18 at 12.41.57 AM.jpeg"
try:
    embedding = extract_embedding(img_path)
    person = identify_person(embedding)
    print(f"✅ Identified as: {person}")
except ValueError as e:
    print(f"⚠️ {e}")

✅ Identified as: Unknown


In [29]:
mtcnn_cpu = MTCNN(image_size=160, margin=14, keep_all=False, device='cpu')
resnet_cpu = InceptionResnetV1(pretrained='vggface2').eval()
torch.save(mtcnn_cpu, r"C:\Users\COMPUMARTS\Downloads\v3\mtcnn.pth")
torch.save(resnet_cpu, r"C:\Users\COMPUMARTS\Downloads\v3\resnet.pth")


C:\Users\COMPUMARTS\anaconda3\envs\myenv\lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_di